# Exercise 09 : Plotly

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import plotly.graph_objects as go

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [3]:
query = """
SELECT uid, timestamp
FROM checker
WHERE uid LIKE 'user_%' AND status = 'ready' AND labname = 'project1'
"""
df = pd.read_sql(query, conn)
conn.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

x_init = np.array([df['date'].min()])
initial_data = []
for uid in df['uid'].unique():
    y_axis = np.array(df[df['uid'] == uid].groupby('date').size().cumsum())
    initial_data.append(go.Scatter(x=x_init, y=y_axis, mode="lines", name=uid))

frames = []
dates = sorted(df['date'].unique())
commits = df.pivot_table(index='date', columns='uid', aggfunc='size', fill_value=0).cumsum()
for i, date in enumerate(dates):
    x_axis = dates[:i + 1]
    curr_data = []
    for col in commits.columns:
        y_axis = commits[col].loc[:date]
        curr_data.append(go.Scatter(x=x_axis, y=y_axis, mode="markers+lines", name=col))
    title = f"Commit Dynamic up to {date}"
    curr_frame = go.Frame(data=curr_data, layout={"title": title})
    frames.append(curr_frame)

figure = go.Figure(
    data=initial_data,
    layout={
        "title": "Commit Dynamic",
        "xaxis": {"title": "Date"},
        "yaxis": {"title": "Cumulative Commits"},
        "updatemenus": [{"type": "buttons", "buttons": [{"method": "animate", "label": "Play", "args": [None]}]}]
    },
    frames=frames
)

figure.show()
